In [1]:
import pandas

In [2]:
emotions = pandas.read_csv('./data/emotions.csv')
emotions

,date,user_id,item_id,type,emoji
0,2023-08-03 21:37:22+03:00,user_21883648,video_2247834,v_top,v_top
1,2023-08-03 21:04:54+03:00,user_24016046,video_2247834,v_top,v_top
2,2023-08-03 20:37:06+03:00,user_20748867,video_22669,v_top,v_top
3,2023-08-03 20:44:12+03:00,user_21545120,video_645471,v_top,v_top
4,2023-08-03 20:45:17+03:00,user_28987830,video_2346806,v_top,v_top
...,...,...,...,...,...
369424,2023-08-02 11:43:43+03:00,user_21597217,video_1180057,pos_emotions,Like
369425,2023-07-25 16:26:35+03:00,user_6805283,video_1180057,pos_emotions,Like
369426,2023-07-28 13:07:10+03:00,user_5419928,video_1180057,pos_emotions,Like
369427,2023-08-07 18:16:30+03:00,user_12044809,video_1180057,pos_emotions,Like


In [3]:
emotions = emotions[emotions['type'] != 'neg_emotions']

In [4]:
reactions = set()
for user_id, item_id in zip(emotions.user_id, emotions.item_id):
    reactions.add((user_id, item_id))

In [5]:
train_data = pandas.read_parquet(
    './data/player_starts_train.parquet', columns = ['date', 'user_id', 'item_id', 'watch_time']
)
train_data

,date,user_id,item_id,watch_time
0,2023-07-21 19:04:50+03:00,user_12964323,video_1042531,51
1,2023-07-21 02:02:41+03:00,user_16517,video_1707159,31
2,2023-07-21 22:00:47+03:00,user_15057892,video_1989987,9
3,2023-07-21 19:09:43+03:00,user_2846972,video_1356486,-1
4,2023-07-21 11:06:58+03:00,user_20517034,video_1380654,11
...,...,...,...,...
69954375,2023-08-21 02:51:53+03:00,user_15478739,video_1449287,291
69954376,2023-08-21 08:40:18+03:00,user_25783543,video_1423321,2
69954377,2023-08-21 05:19:55+03:00,user_3507470,video_464555,261
69954378,2023-08-21 12:32:32+03:00,user_13128840,video_420973,21


In [6]:
train_data = train_data[train_data.date >= "2023-08-01 00:00:00+03:00"]
train_data

,date,user_id,item_id,watch_time
23371198,2023-08-01 11:50:52+03:00,user_18556667,video_15830,51
23371199,2023-08-01 11:47:25+03:00,user_12417386,video_2194155,141
23371200,2023-08-01 11:47:14+03:00,user_12417386,video_2194155,141
23371201,2023-08-01 20:01:00+03:00,user_11892063,video_1770182,451
23371202,2023-08-01 20:20:21+03:00,user_6433209,video_854240,2612
...,...,...,...,...
69954375,2023-08-21 02:51:53+03:00,user_15478739,video_1449287,291
69954376,2023-08-21 08:40:18+03:00,user_25783543,video_1423321,2
69954377,2023-08-21 05:19:55+03:00,user_3507470,video_464555,261
69954378,2023-08-21 12:32:32+03:00,user_13128840,video_420973,21


In [ ]:
from tqdm.auto import tqdm

values = []
for user_id, item_id in tqdm(zip(train_data.user_id, train_data.item_id), total=len(train_data)):
    values.append((user_id, item_id) in reactions)
values

In [ ]:
train_data['reaction'] = values

In [ ]:
train_data['day'] = train_data['date'].apply(lambda x: int(x.split(' ')[0].split('-')[2]))

In [ ]:
import numpy as np

train_data["rank"] = train_data.day.max() - train_data.day.to_numpy()
train_data["rating"] = 100 * 1 / (train_data['rank'] + 1)**2 * np.log(train_data['watch_time']+2) * (1 + 10 * train_data['reaction'])
train_data

In [13]:
popularity = train_data.groupby('item_id')['rating'].sum()
popularity

item_id
video_0            1.744352
video_1           56.761629
video_1000        61.130565
video_10000     5095.179229
video_100000     864.454076
                   ...     
video_999995       0.850299
video_999996       1.821000
video_999997       3.166604
video_999998       0.173287
video_999999       1.145698
Name: rating, Length: 1844876, dtype: float64

In [21]:
const_predict = list(popularity.sort_values(ascending=False).index[:10])
const_predict

['video_283933',
 'video_68646',
 'video_1761620',
 'video_1594159',
 'video_170129',
 'video_2052690',
 'video_803844',
 'video_1508623',
 'video_1210283',
 'video_1508131']

In [22]:
test_user_ids = set(np.load('test_user_ids.npy'))
test_user_ids

{'user_23909185',
 'user_4235297',
 'user_26641195',
 'user_6154739',
 'user_11776568',
 'user_19134421',
 'user_15049768',
 'user_23490712',
 'user_8852409',
 'user_27792003',
 'user_15012727',
 'user_24171720',
 'user_15080762',
 'user_21273357',
 'user_5601539',
 'user_24779428',
 'user_14645425',
 'user_11561740',
 'user_12260731',
 'user_16778190',
 'user_26795506',
 'user_20284925',
 'user_1629611',
 'user_1499651',
 'user_28669758',
 'user_21267267',
 'user_20435024',
 'user_24072700',
 'user_7776',
 'user_5414977',
 'user_12717008',
 'user_5606314',
 'user_11717787',
 'user_5982604',
 'user_17569451',
 'user_19344263',
 'user_6272352',
 'user_15074328',
 'user_19029964',
 'user_2225918',
 'user_27541197',
 'user_222612',
 'user_9638439',
 'user_20071373',
 'user_11007664',
 'user_8457985',
 'user_19940321',
 'user_27051419',
 'user_23922815',
 'user_21276657',
 'user_20234468',
 'user_27844167',
 'user_18786557',
 'user_23359349',
 'user_28108225',
 'user_13323836',
 'user_1429

In [23]:
submission = pandas.read_csv('./submission_0,395.csv').set_index('user_id')
submission

,recs
user_id,
user_26511551,"['video_1545210', 'video_2323123', 'video_9761..."
user_29194819,"['video_283933', 'video_68646', 'video_1508623..."
user_29734049,"['video_283933', 'video_68646', 'video_1508623..."
user_955460,"['video_283933', 'video_68646', 'video_1508623..."
user_7065521,"['video_38143', 'video_1764512', 'video_228142..."
...,...
user_29281681,"['video_283933', 'video_68646', 'video_1508623..."
user_3912848,"['video_2189663', 'video_1095484', 'video_1174..."
user_28389099,"['video_144691', 'video_144691', 'video_144691..."


In [24]:
for user_id in submission.index:
    if user_id not in test_user_ids:
        submission.loc[user_id, 'recs'] = const_predict

In [25]:
submission.to_csv("submission_0,395_const.csv")

In [26]:
submission

,recs
user_id,
user_26511551,"['video_1545210', 'video_2323123', 'video_9761..."
user_29194819,"[video_283933, video_68646, video_1761620, vid..."
user_29734049,"[video_283933, video_68646, video_1761620, vid..."
user_955460,"[video_283933, video_68646, video_1761620, vid..."
user_7065521,"['video_38143', 'video_1764512', 'video_228142..."
...,...
user_29281681,"[video_283933, video_68646, video_1761620, vid..."
user_3912848,"['video_2189663', 'video_1095484', 'video_1174..."
user_28389099,"[video_283933, video_68646, video_1761620, vid..."
